### import the libraries we need

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Get the website

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
source.status_code

200

### Scraping the table from the website and put it into a dataframe

In [4]:
soup = BeautifulSoup(source.content,'lxml')
type(soup)

bs4.BeautifulSoup

In [5]:
table = soup.find_all('table')[0]

In [6]:
df = pd.read_html(str(table))

In [7]:
dataframe = df[0]

In [8]:
dataframe.columns.values 

array([0, 1, 2])

In [9]:
dataframe.columns = dataframe.iloc[0]
dataframe.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [10]:
dataframe = dataframe.reindex(dataframe.index.drop(0))


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [11]:
dataframe = dataframe[dataframe.Borough != 'Not assigned']

In [12]:
print(dataframe.shape)
dataframe.head()

(212, 3)


,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [13]:
dataframe = dataframe.reset_index(drop=True)
print(dataframe.shape)
dataframe.head()

(212, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Combine the rows that has the same postal code, and put the neighbours in the same cell. 

In [14]:
#a = dataframe['Neighbourhood'].groupby(dataframe['Postcode']).apply(lambda x: "{%s}" %', '.join(x))
a = dataframe['Neighbourhood'].groupby([dataframe['Postcode'],dataframe['Borough']]).apply(lambda x: ', '.join(x))
a.head()


Postcode  Borough    
M1B       Scarborough                            Rouge, Malvern
M1C       Scarborough    Highland Creek, Rouge Hill, Port Union
M1E       Scarborough         Guildwood, Morningside, West Hill
M1G       Scarborough                                    Woburn
M1H       Scarborough                                 Cedarbrae
Name: Neighbourhood, dtype: object

In [15]:
df = pd.DataFrame(a).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### If a cell has borough but a Not Assigned neighborhood, then the neighborhood will be the same as the borough

In [16]:
a = df[df.Neighbourhood == 'Not assigned'].index
a

Int64Index([85], dtype='int64')

In [17]:
df.loc[a,'Neighbourhood'] = df.loc[a,'Borough']

### print the number of rows of your dataframe.

In [18]:
df.shape

(103, 3)

# Add Latitude and longitude to the dataframe

In [19]:
df_ = pd.read_csv('/Users/bowenliu/Documents/ibm_data_scientist/capstone_project/Geospatial_Coordinates.csv')

In [20]:
df_.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [21]:
df_.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
df = pd.merge(df,df_)

In [23]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Explore and cluster the neighborhoods in Toronto. 

In [24]:
import numpy as np # library to handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### work with only boroughs that contain the word Toronto

In [166]:
df = df[df['Borough'].str.contains('Toronto')]
df.reset_index()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,47,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


## Create a map of Toronto with neighborhoods superimposed on top

In [26]:
latitude = 43.657952
longitude = -79.387383

In [27]:
# create map of toronto using latitude and longitude values

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Define Foursquare Credentials and Version

In [28]:
CLIENT_ID = 'P1E2EGPIORWJ2JCIUBTJ42T41UGN1YSGGJGQONGBJMPZREBX' # your Foursquare ID
CLIENT_SECRET = 'D1PT5043NRXZFD2MYHDWLAAQOBF2IWCL5UQ0AKWEVFNTAFI4' # your Foursquare Secret
VERSION = '20181124' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P1E2EGPIORWJ2JCIUBTJ42T41UGN1YSGGJGQONGBJMPZREBX
CLIENT_SECRET:D1PT5043NRXZFD2MYHDWLAAQOBF2IWCL5UQ0AKWEVFNTAFI4


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [29]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a new dataframe called toronto_venues that includes Venues information

In [30]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [167]:
print(toronto_venues.shape)
toronto_venues.head(5)

(1713, 6)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
The Beaches,43.676357,-79.293031,Crazy Sexy Cool,43.680391,-79.290656,Boutique
"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


#### Let's find out how many unique categories can be curated from all the returned venues

In [32]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} unique Neighbourhood'.format(len(toronto_venues['Neighborhood'].unique())))

There are 238 unique categories.
There are 38 unique Neighbourhood


## Analyze Each Neighbourhood

In [33]:
#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'],prefix="",prefix_sep="")

#add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood']
print(toronto_onehot.shape)
#move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head()

(1713, 239)
(1713, 239)


,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Let's find out all the neighbour where therea are at leat one coffee shop/cafe nearby.

In [89]:
neigh_grouped = toronto_onehot.groupby('Neighbourhood').sum().reset_index()
#print(neigh_grouped.shape)
#neigh_grouped.head()
conditions = neigh_grouped[['Café','Coffee Shop']].sum(axis=1) > 1
neigh_coffee = neigh_grouped[conditions]


neigh_coffee.shape
neigh_coffee.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",1,0,0,0,0,0,0,0,0,4,0,0,0,0,1,1,0,2,0,0,0,0,2,0,3,0,0,0,0,0,0,0,0,0,0,2,0,1,3,0,0,1,2,1,0,0,5,0,0,0,0,0,0,0,3,0,7,0,0,0,0,0,2,0,2,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,1,0,1,0,4,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,2,0,1,0,0,1,0,0,1,0,3,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,4,0,0,2,0,0,0,0,0,0,4,1,0,0,0,1,0,0,0,1,0,0,1,0
1,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,2,0,0,0,1,1,

##### so there are 26 neighbours out of 38 that has at least one coffee shop/cafe nearby.
##### let's exam each neighbourhood and find out the top 5 category of venues of those neighbours. 

In [56]:
t = neigh_coffee.drop(['Neighbourhood'],axis=1,inplace=False)

t=t.sum(axis=0).sort_values(ascending=False)
t.head(10)


Coffee Shop            152
Café                    95
Restaurant              53
Hotel                   40
Bar                     39
Italian Restaurant      38
Bakery                  34
Pizza Place             33
American Restaurant     28
Japanese Restaurant     28
dtype: int64

#### Average coffee shop and cafes for each neghbourhood

In [37]:
coffee_shop_number = t['Coffee Shop']
cafe_number = t['Café']
average_coffee_per_neighbourhood = (coffee_shop_number + cafe_number) / neigh_coffee.shape[0]
print(average_coffee_per_neighbourhood)

9.5


#### so from above analysis, we can find the top 5 categories of venues near an existing coffee shop/cafe are: resturants, hotel, bar,bakery, and pizza place. 

#### Next we are going to find those neighbours with the following conditons:
#### 1. the number of coffee shops is less than 9
#### 2. the neighbours has the top 5 categories of venues. 

In [38]:
conditions = neigh_grouped[['Café','Coffee Shop']].sum(axis=1) < 9
neigh_target = neigh_grouped[conditions]
print(neigh_target.shape)
neigh_target.head()



(27, 239)


,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
1,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,2,0,0,0,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,0,0,1,3,4,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,3,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Brockton, Exhibition Place, Parkdale Village",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [39]:
toronto_venues.set_index('Neighborhood',inplace=True)

In [40]:
b = toronto_venues.loc[neigh_target['Neighbourhood']]
b.reset_index(inplace=True)
print(b.columns.values)

['Neighborhood' 'Neighborhood Latitude' 'Neighborhood Longitude' 'Venue'
 'Venue Latitude' 'Venue Longitude' 'Venue Category']


In [41]:

c = b['Venue Category'].groupby([b['Neighborhood'],b['Neighborhood Latitude'],b['Neighborhood Longitude']]).apply(lambda x: ', '.join(x))


In [42]:
e = pd.DataFrame(c)
print(e.shape)
e.head()

(27, 1)


,,,Venue Category
Neighborhood,Neighborhood Latitude,Neighborhood Longitude,
Berczy Park,43.644771,-79.373306,"Liquor Store, Steakhouse, Concert Hall, Museum..."
"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,"Gym, Coffee Shop, Bar, Caribbean Restaurant, C..."
Business reply mail Processing Centre969 Eastern,43.662744,-79.321558,"Brewery, Farmers Market, Comic Shop, Pizza Pla..."
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",43.628947,-79.394420,"Airport, Airport Lounge, Harbor / Marina, Airp..."
"Cabbagetown, St. James Town",43.667967,-79.367675,"Diner, Italian Restaurant, Indian Restaurant, ..."


#### Let's find out those neighborhoods which has the top 5 venues

In [118]:
mylist = ['Restaurant','Bar','Hotel','Pizza Place','Bakery']
for i in mylist:
    f = e[e['Venue Category'].str.contains(i)]
print(f.shape)
f

(8, 1)


,,,Venue Category
Neighborhood,Neighborhood Latitude,Neighborhood Longitude,
Berczy Park,43.644771,-79.373306,"Liquor Store, Steakhouse, Concert Hall, Museum..."
"Cabbagetown, St. James Town",43.667967,-79.367675,"Diner, Italian Restaurant, Indian Restaurant, ..."
"Dovercourt Village, Dufferin",43.669005,-79.442259,"Bar, Middle Eastern Restaurant, Supermarket, G..."
"Harbord, University of Toronto",43.662696,-79.400049,"Japanese Restaurant, Restaurant, Italian Resta..."
"High Park, The Junction South",43.661608,-79.464763,"Bar, Italian Restaurant, Park, Gastropub, Spea..."
"Little Portugal, Trinity",43.647927,-79.419750,"Brewery, Pizza Place, Korean Restaurant, Ice C..."
Studio District,43.659526,-79.340923,"Ice Cream Shop, Sandwich Place, Coffee Shop, F..."
"The Danforth West, Riverdale",43.679557,-79.352188,"Greek Restaurant, Ice Cream Shop, Cosmetics Sh..."


#### above all, those are the 8 locations that is good for setting up a coffee shop/cafe business. 
#### now we are going to check among these 8 neighbourhood, which one has the least coffee shop/cafes. 

In [119]:
ff=f.reset_index(inplace=False)


In [168]:
yy=neigh_grouped.set_index('Neighbourhood',inplace=False)
aa=yy.loc[ff['Neighborhood'],['Coffee Shop','Café']]
aa

,Coffee Shop,Café
Neighbourhood,,
Berczy Park,4,2
"Cabbagetown, St. James Town",4,2
"Dovercourt Village, Dufferin",0,1
"Harbord, University of Toronto",2,3
"High Park, The Junction South",0,2
"Little Portugal, Trinity",3,4
Studio District,3,4
"The Danforth West, Riverdale",3,1


In [169]:
bb = aa.sum(axis=1).sort_values(ascending=True)
bb

Neighbourhood
Dovercourt Village, Dufferin      1
High Park, The Junction South     2
The Danforth West, Riverdale      4
Harbord, University of Toronto    5
Berczy Park                       6
Cabbagetown, St. James Town       6
Little Portugal, Trinity          7
Studio District                   7
dtype: int64

In [153]:
the_best_location = bb.index[0]
print("the best location to start a coffee shop business is {} ".format(the_best_location))
a = f.loc[the_best_location]

the best location to start a coffee shop business is Dovercourt Village, Dufferin 


In [154]:
a.reset_index(inplace=True)

In [164]:
b = a.iloc[0].values[0:2].tolist()

In [165]:
print("the best location to start a coffee shop business is {} ".format(the_best_location))
print("the location is {}".format(b))

the best location to start a coffee shop business is Dovercourt Village, Dufferin 
the location is [43.66900510000001, -79.4422593]
